##US Mass Shooting Analysis  
**Gun Violence, USA**  


Gun violence in the United States is a complex and contentious issue that has garnered significant attention from lawmakers, researchers, and the public in recent years. The proliferation of firearms in the country has led to a high number of gun-related deaths and injuries, with tragic incidents of mass shootings occurring with alarming frequency. This data report aims to provide a comprehensive analysis of the trends and patterns of gun violence in the US, using the latest available data from reliable sources. By examining key indicators such as the number of gun deaths and injuries, demographics of victims and perpetrators, and legislative responses to gun violence, this report seeks to shed light on the scope and impact of this issue and offer insights for potential solutions.

Analysis conducted by Frank Ebere referencing data from Kaggle https://www.kaggle.com/datasets/nidzsharma/us-mass-shootings-19822023






**Part 1 - Data Cleaning**

In [1]:


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go

# READING MY df FROM THE DESKTOP
"""
df = pd.read_csv('/content/shooting_data.csv')
df2 = pd.read_csv('/content/clean_data.csv')
df.head()
"""


from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/shooting_data.csv')
df2 = pd.read_csv('/content/drive/MyDrive/clean_data.csv')


Mounted at /content/drive


In [2]:
#Seperate data- 'Location' contains City/State using Split

split_col = df['location'].str.split(',', expand =True)

df['City'] = split_col[0]
df['State'] = split_col[1]

In [3]:
# CONVERT THE DATE DATA TYPE AND CREATE A NEW COLUMN FOR IT.

#d show df.head()

df['date'] = pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['quarter'] = df['date'].dt.quarter
df['half'] = df['date'].dt.month.apply(lambda x: 1 if x <= 6 else 2)
df['month_name'] = df['date'].dt.month_name()
df['day_of_week'] = df['date'].dt.day_name()
df.head()

<ipython-input-3-9cb595204746>:5: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['date'] = pd.to_datetime(df['date'])


,case,location,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,...,latitude,longitude,type,year,City,State,quarter,half,month_name,day_of_week
0,Nashville religious school shooting,"Nashville, TN",2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,...,-,-,Mass,2023,Nashville,TN,1,1,March,Monday
1,Michigan State University shooting,"East Lansing, Michigan",2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,...,-,-,Mass,2023,East Lansing,Michigan,1,1,February,Monday
2,Half Moon Bay spree shooting,"Half Moon Bay, California",2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,workplace,67,-,...,-,-,Spree,2023,Half Moon Bay,California,1,1,January,Monday
3,LA dance studio mass shooting,"Monterey Park, California",2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,...,-,-,Mass,2023,Monterey Park,California,1,1,January,Saturday
4,Virginia Walmart shooting,"Chesapeake, Virginia",2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,...,-,-,Mass,2022,Chesapeake,Virginia,4,2,November,Tuesday


In [4]:
# COUNTING THE NUMBER OF ROWS IN THE DATASET

len(df)



141

In [5]:
#Replace '-' in column age_of_shooter
#replace('(TK-"fewer than 10"','0') in column Injured

df ['injured'] = df['injured'].replace('(TK-"fewer than 10"',0)
df ['age_of_shooter'] = df['age_of_shooter'].replace('-', 0)
df ['total_victims'] = df ['total_victims'].replace('TK', 0)
df ['latitude'] = df ['latitude'].replace('-', 0.00)
df ['longitude'] = df ['longitude'].replace('-', 0.00)
df ['mental_health_details'] = df ['mental_health_details'].replace('-', 'PENDING COLLECTION')





In [6]:
# GENDER SPECIFICATION COUNT AND ALSO REPLACING DESCREPANCES INN THE DATA

df ['gender'] = df ['gender'].replace('M', 'Male')
df ['gender'] = df ['gender'].replace('F', 'Female')
df ['gender'] = df ['gender'].replace('F (identified as transgender, per police)', 'Female')


df['gender'].value_counts()

Male             135
Female             4
Male & Female      2
Name: gender, dtype: int64

In [7]:
#WEAPONS USED IN THESE SHOOTINGS, IF THEY WERE OBTAINED LEGALLY OR NOT

df ['weapons_obtained_legally'] = df ['weapons_obtained_legally'].replace({'yes': 'Yes','-': 'TBD', 'yes': 'Yes',
                                                                           'Unknown': 'TBD', '\nYes': 'Yes',
                                                                           'Kelley passed federal criminal background checks; the US Air Force failed to provide information on his criminal history to the FBI': 'Yes',
                                                                           'Yes ("some of the weapons were purchased legally and some of them may not have been")': 'Yes'})



df['weapons_obtained_legally'].value_counts()

Yes     94
TBD     30
No      16
Yes      1
Name: weapons_obtained_legally, dtype: int64

In [8]:


df['race'] = df['race'].replace({'white' : 'White','black' : 'Black', 'White ' : 'White',
                                 '-': 'Unclear', 'unclear' : 'Unclear', 'Other' : 'OTHER'})

df['race'].value_counts()

White              73
Black              25
Unclear            14
Latino             11
Asian              10
OTHER               5
Native American     3
Name: race, dtype: int64

In [9]:
df['location.1'] = df['location.1'].replace({'Other\n' : 'Other', '\nWorkplace': 'Workplace', 
                                             'workplace' : 'Workplace','religious': 'Religious' })

df['location.1'].value_counts()

Other        53
Workplace    51
School       22
Religious     8
Military      6
Airport       1
Name: location.1, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   case                              141 non-null    object        
 1   location                          141 non-null    object        
 2   date                              141 non-null    datetime64[ns]
 3   summary                           141 non-null    object        
 4   fatalities                        141 non-null    int64         
 5   injured                           141 non-null    int64         
 6   total_victims                     141 non-null    object        
 7   location.1                        141 non-null    object        
 8   age_of_shooter                    141 non-null    object        
 9   prior_signs_mental_health_issues  141 non-null    object        
 10  mental_health_details             141 non-null    

In [11]:
df ['where_obtained'] = df ['where_obtained'].replace({'-': 'TBD', 'Unknown': 'TBD', 'Unclear': 'TBD'})


df ['where_obtained'].value_counts()

TBD                                                            68
Purchased from an individual                                    2
Glock and Remington stolen from grandfather                     1
Trader's Village flea market in Grand Prairie, Texas            1
Hunting Supplies of Hawaii (The Armory) in Honolulu, Hawaii     1
                                                               ..
Florida Gun Center                                              1
Sharpshooters Small Arms Range                                  1
Local gun store in Killeen, Guns Galore                         1
Gun was his father's.                                           1
Garcia Gun Center in Hialeah, Fla.                              1
Name: where_obtained, Length: 73, dtype: int64

In [12]:
#split_col = df['weapon_type']

df ['weapon_type'] = df ['weapon_type'].replace({' ': 'UNIDENTIFIED', 'One semiautomatic handgun':'semiautomatic handgun',
                                                 'handgun\n':'handgun',
                                                 'Two handguns\n':'handgun', 'semiautomatic handgun\n':'semiautomatic handgun', 
                                                 'One semiautomatic handgun,revolver':'One revolver','2 assault rifles; semiautomatic handgun\n':'assault rifles'})

df ['weapon_type'].value_counts()

semiautomatic handgun                                   34
semiautomatic rifle                                     12
handgun                                                  8
Two semiautomatic handguns                               6
One semiautomatic handgun, one revolver                  4
                                                        ..
assault rifles                                           1
Three semi-automatic handguns and two hunting knives     1
Two handguns and a butcher knife                         1
9mm semi-automatic handgun                               1
One shotgun                                              1
Name: weapon_type, Length: 70, dtype: int64

In [13]:
# Define the age ranges and labels
df['age_of_shooter'] = df['age_of_shooter'].replace({'-': 0})

# CONVERT THE DATA TYPE FROM SERIES TO INTERGER
df['age_of_shooter'] = df['age_of_shooter'].astype(int)

# LABEL THE AGE GROUPS AND ESTABLISH AGE RANGES
age_ranges = [0, 10, 18, 39, 59, 120]
age_labels = ['Unknown', 'Teenage', 'Early Adulthood', 'Middle Adulthood', 'Old Age']

# Use the cut() function to categorize the age groups
df['age_group'] = pd.cut(df['age_of_shooter'], bins=age_ranges, labels=age_labels)

df['age_group'].value_counts()

Early Adulthood     79
Middle Adulthood    43
Teenage             11
Old Age              6
Unknown              0
Name: age_group, dtype: int64

In [14]:
df.describe()

,fatalities,injured,age_of_shooter,year,quarter,half
count,141.000000,141.000000,141.000000,141.000000,141.000000,141.000000
mean,7.808511,11.205674,33.531915,2010.382979,2.404255,1.460993
std,7.463162,46.579505,13.726696,10.796600,1.140042,0.500253
min,3.000000,0.000000,0.000000,1982.000000,1.000000,1.000000
25%,4.000000,1.000000,22.000000,2005.000000,1.000000,1.000000
50%,6.000000,3.000000,32.000000,2014.000000,2.000000,1.000000
75%,8.000000,10.000000,43.000000,2018.000000,3.000000,2.000000
max,58.000000,546.000000,72.000000,2023.000000,4.000000,2.000000


##Part II    
Showing graphical analiysis from post shooting data as clean Data.

In [15]:




df = df.drop('location', axis= 1)

df.to_csv(r'/content/clean_data.csv', index = False)

df2 = pd.read_csv('/content/drive/MyDrive/clean_data.csv')

df2.head()




,case,date,summary,fatalities,injured,total_victims,location.1,age_of_shooter,prior_signs_mental_health_issues,mental_health_details,...,longitude,type,year,City,State,quarter,half,month_name,day_of_week,age_group
0,Nashville religious school shooting,2023-03-27,"Audrey Hale, 28, who was a former student at t...",6,1,6,School,28,-,PENDING COLLECTION,...,0.0,Mass,2023,Nashville,TN,1,1,March,Monday,Early Adulthood
1,Michigan State University shooting,2023-02-13,"Anthony D. McRae, 43, opened fire at Berkey Ha...",3,5,8,School,43,-,PENDING COLLECTION,...,0.0,Mass,2023,East Lansing,Michigan,1,1,February,Monday,Middle Adulthood
2,Half Moon Bay spree shooting,2023-01-23,"Chunli Zhao, 67, suspected of carrying out the...",7,1,8,Workplace,67,-,PENDING COLLECTION,...,0.0,Spree,2023,Half Moon Bay,California,1,1,January,Monday,Old Age
3,LA dance studio mass shooting,2023-01-21,"Huu Can Tran, 72, fled the scene in a white va...",11,10,21,Other,72,yes,"According to the LA Times, ""Two law enforcemen...",...,0.0,Mass,2023,Monterey Park,California,1,1,January,Saturday,Old Age
4,Virginia Walmart shooting,2022-11-22,"Andre Bing, 31, who worked as a supervisor at ...",6,6,12,Workplace,31,-,PENDING COLLECTION,...,0.0,Mass,2022,Chesapeake,Virginia,4,2,November,Tuesday,Early Adulthood


In [16]:
grouped_by_fatalities = df.groupby('case')['fatalities'].sum() #This is used to sum the count of fatalities

top_10_by_fatalities = grouped_by_fatalities.sort_values(ascending=False).head(10)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=top_10_by_fatalities.values,
    y=top_10_by_fatalities.index,
    orientation='h',
    text=top_10_by_fatalities.values,
    textposition='auto',
))
fig.update_layout(
    title='Top 10 Cases by fatalities',
    xaxis_title='Case',
    yaxis_title='Fatalities',
    height=600,
    margin=dict(l=0, r=0, t=50, b=0),
    yaxis=dict(autorange="reversed")
)
fig.show();

* This step identifies which age bracket have mostly been involved in mass shootings between 2012 to 2020.

In [ ]:
! pip install powerbiclient